In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# =============================================================================
# ** == STILL TO DO
# =============================================================================

# Python Imports
from datetime import datetime
import h5py
import numpy as np
import glob
import os
import pandas as pd

# matlab Imports
import matlab.engine

# Reading the Experiment Data sheet
from docx import Document

# NWB imports
from pynwb import NWBFile, get_manager
from pynwb import TimeSeries
from pynwb import NWBHDF5IO

# my scripts
#import from_ini # this has crashed for some reason?????
#print(os.listdir(session_date_folder)) gets list of files in directory
'''
this will identify an error but give it an exception, may need it later

try:
    sbj = pd.read_excel(data_sheet, sheet_name= i , index_col=0)
except ValueError:
    continue
'''

/Users/solomia/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'\nthis will identify an error but give it an exception, may need it later\n\ntry:\n    sbj = pd.read_excel(data_sheet, sheet_name= i , index_col=0)\nexcept ValueError:\n    continue\n'

In [3]:
import matlab.engine
eng = matlab.engine.start_matlab()

the MATLAB function has been cancelled


In [11]:
a, b = eng.edit_neurosee("smb://bg-thefarm-2012.bg.ic.ac.uk/Schultz_group_data/Crazy Eights/Ann/GCaMP6 imaging/2018.03.07/2P/20180307_17_09_31_2P/20180307_17_09_31_2P_XYT.tif")
print(a, b)

MatlabExecutionError: 
  File /Applications/MATLAB_R2017b.app/toolbox/matlab/imagesci/private/getFileFromURL.m, line 19, in getFileFromURL

  File /Applications/MATLAB_R2017b.app/toolbox/matlab/imagesci/imfinfo.m, line 84, in imfinfo

  File /Users/solomia/Dropbox/2p_da/nwb_scripts/edit_neurosee.m, line 25, in edit_neurosee
Can't read URL "smb://bg-thefarm-2012.bg.ic.ac.uk/Schultz_group_data/Crazy Eights/Ann/GCaMP6 imaging/2018.03.07/2P/20180307_17_09_31_2P/20180307_17_09_31_2P_XYT.tif".


In [205]:
def sbj_sheet_list(datas_dir):
    data_sheet = pd.ExcelFile(datas_dir)
    sheets = data_sheet.sheet_names
    return(sheets)

def get_exp_data(data_sheet):
#    sheets = open_excel_sheet(datas_dir)
    exp_details = pd.read_excel(data_sheet, sheet_name= 'Experiment Details', index_col=0)
    return(exp_details)

def create_mouse_id_folder(mouse_id):
    """ creates folder with mouse ID """
#    this is a clean way to join a path name (do this everywhere)
    target_dir = os.path.join('/Users/solomia/Dropbox/2p_da/data_files/nwb_folder/', mouse_id)
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    return target_dir

def create_mouse_date_folder(mouse_id, exp_date):
    mouse_target_dir = create_mouse_id_folder(mouse_id)
    target_dir = os.path.join(mouse_target_dir, exp_date)
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    return target_dir

def is_sbj_sheet(sheet):
    if 'template' in sheet.lower():
        return False
    if sheet.lower().startswith('subject details'):
        return True
    return False

# returns the first folder of the 2p data for analysis
# what if this ran the ROI and timeseries extraction together??
# need to iterate through starting as index first folder
def corresponding_2P(imaging_folder, exp_date, imaging_times):
    twop_folders = sorted(os.listdir(imaging_folder))
    first_folder = exp_date + '_' + imaging_times.index[0].strftime('%H_%M_%S') + '_2P'
    if first_folder in twop_folders:
        twop_raw = os.path.join(first_folder, first_folder + '_XYT.raw')
        print(twop_raw)
        for i in twop_folders:
            print()

In [206]:
def get_data(data_sheet, imaging_folder, behav_folder):
    sheets = sbj_sheet_list(data_sheet)
    # using the square brackets you can construct paths easier (look into this)
    sbj_details = [x for x in sheets if is_sbj_sheet(x)]
    for i in sbj_details:
        sbj = pd.read_excel(data_sheet, sheet_name= i , index_col=0)
        times = sbj.loc['Time':].copy()
        times.columns = times.iloc[0]
        times.drop(['Time'],inplace=True)
        
#        This returns a check on whether there is a TRUE inside the column
        imaging_times = times[times['2P (X)'].notnull()]
        
        if len(imaging_times) > 0:
            mouse_id = sbj.loc['Number','Unnamed: 1']
            exp_data = get_exp_data(data_sheet)
            exp_date = exp_data.at['Date','Unnamed: 1'].replace('.', '')
            mouse_date_folder = create_mouse_date_folder(mouse_id, exp_date)
           
# looking for nwb files
            session_start = str(imaging_times.index[0])[0:5]
            output_file_name = mouse_id + '_' + exp_date + \
            '_' +  session_start.replace(':','_') + '.nwb'
            output_filepath = os.path.join(mouse_date_folder, output_file_name)
            if not os.path.exists(output_filepath):
                nwb_file = NWBFile(
                            source = '',
                            session_description = mouse_id + ' ' + exp_date + ' ' + session_start,
                            identifier = mouse_id + ' ' + exp_date + ' ' + session_start, 
                            session_start_time = session_start,
                #            file_create_date = datetime.now(tz=None),
                #            experimenter='Dr Ann Go',
                            lab = 'Neural Coding',
                            institution = 'Imperial College London',
                            experiment_description = 'GCamp Imaging',
                            virus = sbj.index[6]+':'+sbj.loc['GCaMP Inj','Unnamed: 1']+', '\
                            +sbj.index[7]+':'+sbj.loc['Methoxy Inj','Unnamed: 1']
                #            session_id='test_001',
                            )  
                
                #find folder coressponding to 2P data
                #so session start should match
                first_2P_path = corresponding_2P(imaging_folder,exp_date,imaging_times)
#           INPUT THIS TO ROI CODE, then TIMESERIES
# TAKE OUTPUT OF THIS INTO THE TIMESERIES BUILDING!!!!!!!!!!

                data = list(range(100, 200, 10))
                timestamps = list(range(10))
                red_ts = TimeSeries('red_timeseries', #this names the TS in the epoch 
                                     'PyNWB tutorial', 
                                     data, 
                                     'SIunit', 
                                     starting_time=0.0, 
                                     rate=1.0)
                green_ts = TimeSeries('green_timeseries', #this names the TS in the epoch 
                                     'PyNWB tutorial', 
                                     data, 
                                     'SIunit', 
                                     starting_time=0.0, 
                                     rate=1.0)
                #nwb_file.add_acquisition(test_ts)
                # test data
                data = list(range(0, 100, 10))
                timestamps = list(range(10))
                
                nwb_file.create_epoch('the first epoch',' ', 2.0, 4.0, [red_ts, green_ts],'description')
                nwb_file.set_epoch_timeseries('the first epoch',red_ts)
                nwb_file.set_epoch_timeseries('the first epoch',green_ts)
        
                file = NWBHDF5IO(output_filepath, manager=get_manager(), mode='w')
                file.write(nwb_file)
                file.close()
            else:
                io = NWBHDF5IO(output_filepath)
                nwbfile = io.read()
                find_epoch = nwbfile.get_epochs()
                print(find_epoch)

In [207]:
get_data('/Users/solomia/Dropbox/2p_da/data_files/2018.03.07/Experiment Data Sheet - 2018.03.07.xlsm','/Users/solomia/Dropbox/2p_da/data_files/2018.03.07/2P',' ')

20180307_15_55_00_2P/20180307_15_55_00_2P_XYT.raw
